In [1]:
import sys  
sys.path.insert(0, '/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/')

In [2]:
import src.utils
from src.utils import import_dta, import_tracks_dta, successful_creators_followers,\
gen_active_relations, get_fan_interactions_per_week, calculate_avg_monthly_valence,\
gen_active_relations_prob, get_fan_interactions_per_week_prob
import datetime
import pandas as pd

In [3]:
path_dir = r"/Users/../Volumes/Raw/"

In [18]:
#we will skip comments for now
import datetime
#affiliations :follows
#favoritings :likes

follows_sent = import_dta(path_dir, "12sample_affiliations_sent.dta")
comments_sent = import_dta(path_dir, "12sample_comments_made.dta")
shares_sent = import_dta(path_dir, "12sample_reposts_made.dta")
likes_sent = import_dta(path_dir, "12sample_favoritings_made.dta")

user_info = import_dta(path_dir, "12sample_user_infos.dta")
df13 = import_tracks_dta(path_dir, "12sample_tracks.dta")


########
follows_received = import_dta(path_dir, "12sample_affiliations_received.dta")
comments_received = import_dta(path_dir, "12sample_comments_received.dta")
shares_received = import_dta(path_dir, "12sample_reposts_received.dta")
likes_received = import_dta(path_dir, "12sample_favoritings_received.dta")

%%%%%%%%%% 12sample_affiliations_sent.dta %%%%%%%%%%
     fan_id  contact_id          created_at
0  37825116       61046 2013-03-01 00:14:58
1  37825116       80377 2013-03-01 00:14:58
2  37825116      122596 2013-03-01 00:14:58
3  37825116      133935 2013-03-01 00:14:58
4  37825116      165779 2013-03-01 00:14:58
(800913, 3)
Index(['fan_id', 'contact_id', 'created_at'], dtype='object')
%%%%%%%%%% 12sample_comments_made.dta %%%%%%%%%%
    user_id  track_id  owner_id          created_at
0  38303468    400184    170967 2014-02-28 05:38:50
1  38303468    400184    170967 2014-02-28 05:29:57
2  38469256    667083    142644 2013-07-11 22:39:39
3  38386425    667083    142644 2013-03-06 18:26:29
4  38274244   1110211      6289 2013-04-24 16:36:28
(29258, 4)
Index(['user_id', 'track_id', 'owner_id', 'created_at'], dtype='object')
%%%%%%%%%% 12sample_reposts_made.dta %%%%%%%%%%
   reposter_id          created_at  song_id  owner_id
0     38343678 2014-06-06 20:44:06   354983    152008
1     38

In [19]:
follows_received['inbound_activity'] = 'follow'
follows_received.columns = ['fan_id', 'user_id', 'date_sent', 'inbound_activity']

if 'song_id' in shares_received:
    shares_received.drop(columns=["song_id"])
shares_received = shares_received[['reposter_id', "owner_id", 'created_at']]
shares_received['inbound_activity'] = 'share'
shares_received.columns = ['fan_id', 'user_id', 'date_sent', 'inbound_activity']

if 'track_id' in likes_received:
    likes_received = likes_received.drop(columns=["track_id"])
likes_received['inbound_activity'] = 'like'
likes_received.columns = ['fan_id', 'user_id', 'date_sent', 'inbound_activity']

if 'track_id' in comments_received:
    comments_received = comments_received.drop(columns=["track_id"])
comments_received['inbound_activity'] = 'comment'
comments_received.columns = ['fan_id', 'user_id', 'date_sent', 'inbound_activity']


df = pd.concat([follows_received, shares_received, likes_received, comments_received])
df = df.merge(user_info, left_on='user_id', right_on='user_id')
# df = df.loc[df['type'] != 'creator', :]
df['interaction_week'] = df.date_sent.apply(lambda x: x.week)
df = df.loc[df.fan_id != df.user_id, :]

In [30]:
month = 12
delta_M = datetime.timedelta(days=month * 30)
delta = datetime.timedelta(days=30)
date = datetime.datetime(2013, 3, 1, 0, 0, 0) + delta_M

mask1 = (df['date_sent'] >= (date)) & (df['date_sent'] <= (date + delta))

df_f = df[mask1]

In [24]:
min(df_f.date_sent)
max(df_f.date_sent)

Timestamp('2013-06-29 00:01:15')